In [2]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os
import warnings
warnings.filterwarnings('ignore')

# # Load the cleaned data file
# cwd = os.getcwd()
# print(cwd)
# parent_dir = os.path.dirname(cwd)
# print(parent_dir)
filename = r'C:\Users\jbh\Desktop\NYPD_Complaint_Data_Cleaned.csv'
df = pd.read_csv(filename)


# df.shape

C:\Users\jbh\AppData\Local\Temp\ipykernel_23928\3090426702.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Focus crimes: FELONY ASSAULT, ROBBERY, RAPE
focus_crimes = [
    'HARRASSMENT 2', 
    'ROBBERY', 
    'ASSAULT 3 & RELATED OFFENSES', 
    'FELONY ASSAULT', 
    'PETIT LARCENY', 
    'GRAND LARCENY',
    'SEX CRIMES',
    'RAPE'
]


df_focus = df[df['Offense_Description'].isin(focus_crimes)]


# Keep columns: Complaint_From_Date, Complaint_From_Time, Lattitude, Longtitude, Victim_Sex
df_focus = df_focus[['Complaint_From_Date', 'Complaint_From_Time', 'Latitude', 'Longitude', 'Victim_Sex', 'Offense_Description']]

# Drop rows where Victim_Sex is not "M" or "F" (D=Business/Organization and E = PSNY/People of the State of New York which is not relevant to the case)
df_focus = df_focus[df_focus['Victim_Sex'] != 'D']
df_focus = df_focus[df_focus['Victim_Sex'] != 'E']
# The footnotes doesn't provide any useful information on the Victim_Sex == "L". We think it might be "Lady" or "LQBTQ" but we are not sure. So we will drop these rows. 
# In this subset of data, they only account for approx 800 rows out of 800k rows. So it's not a big deal to drop them.
df_focus = df_focus[df_focus['Victim_Sex'] != 'L']


# convert to datetime
df_focus['Complaint_From_Date'] = pd.to_datetime(df_focus['Complaint_From_Date'])
# create a column "DayOfWeek" and remove the column "Complaint_From_Date"
df_focus['weekday'] = df_focus['Complaint_From_Date'].dt.day_name()

# create a column "year" with the year of the date
df_focus['year'] = df_focus['Complaint_From_Date'].dt.year

df_focus = df_focus[df_focus['year'] >= 2019]

df_focus = df_focus.drop(columns=['Complaint_From_Date'])

# convert Complaint_From_Time to datetime
df_focus['Complaint_From_Time'] = pd.to_datetime(df_focus['Complaint_From_Time'], format='%H:%M:%S').dt.time
# create a column "hour" and remove the column "Complaint_From_Time"
df_focus['hour'] = pd.to_datetime(df_focus['Complaint_From_Time'], format='%H:%M:%S').dt.hour
df_focus = df_focus.drop(columns=['Complaint_From_Time'])

# loop through all unique Offense_Descriptions and assign it an arbitrary number
offense_description_dict = {}
for i, offense_description in enumerate(df_focus['Offense_Description'].unique()):
    offense_description_dict[offense_description] = i

# save the dictionary to a json file
with open('offense_description_dict_usecase1.json', 'w') as f:
    json.dump(offense_description_dict, f)

# map the Offense_Description to the arbitrary number
df_focus['Offense_Description'] = df_focus['Offense_Description'].map(offense_description_dict)

# rename Offense_Description to "crime_id"
df_focus = df_focus.rename(columns={'Offense_Description': 'crime_id'})

# loop through weekday and assign number from 0 to 6
weekday_dict = {}
for i, weekday in enumerate(df_focus['weekday'].unique()):
    weekday_dict[weekday] = i

# map the weekday to the arbitrary number
df_focus['weekday'] = df_focus['weekday'].map(weekday_dict)

# rename weekday to "day"
df_focus = df_focus.rename(columns={'weekday': 'day'})

# rename Victim_Sex to "sex"
df_focus = df_focus.rename(columns={'Victim_Sex': 'sex'})

# convert latitude and longitude to float
df_focus['Latitude'] = df_focus['Latitude'].astype(float)
df_focus['Longitude'] = df_focus['Longitude'].astype(float)

# rename to "lat" and "lon"
df_focus = df_focus.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'})

# only keep 10k rows for now
# df_focus = df_focus.sample(n=10000)

# save the df to json with comma separated entries encapulated in square brackets
df_focus.to_json('DataUseCase1-v2.json', orient='records', lines=False)

df_focus

# REMEMBER TO UPDATE THE HTML FOR USECASE1 WITH CHANGES MADE HERE

,lat,lon,sex,crime_id,day,year,hour
62,40.733512,-74.003532,F,0,0,2020,19
63,40.838194,-73.912268,M,1,0,2020,14
64,40.872153,-73.866141,F,2,1,2020,19
65,40.681003,-73.990213,F,0,2,2021,8
66,40.801754,-73.931203,M,1,3,2021,5
...,...,...,...,...,...,...,...
8170547,40.738164,-73.860752,F,0,2,2022,5
8170551,40.739659,-73.774110,M,0,2,2022,14
8170552,40.685418,-73.730032,M,6,4,2020,0
8170553,40.598482,-73.757091,F,0,5,2022,15


In [14]:
# # focus crimes use case 2
# # busines owner wants to avoid all crimes towards businesses, i.e. Victim_Sex == "D"
df_focus2 = df

# Keep columns: Complaint_From_Date, Complaint_From_Time, Lattitude, Longtitude, Victim_Sex
df_focus2 = df_focus2[['Complaint_From_Date', 'Latitude', 'Longitude', 'Victim_Sex', 'Offense_Description']]

# keep rows where Victim_Sex is "D" (Business/Organization)
df_focus2 = df_focus2[df_focus2['Victim_Sex'] == 'D']

# drop Victim_Sex
df_focus2 = df_focus2.drop(columns=['Victim_Sex'])

# convert to datetime
df_focus2['Complaint_From_Date'] = pd.to_datetime(df_focus2['Complaint_From_Date'])
# create a column "Year" and remove the column "Complaint_From_Date"
df_focus2['Year'] = df_focus2['Complaint_From_Date'].dt.year
df_focus2 = df_focus2.drop(columns=['Complaint_From_Date'])

# set latitude and longitude to float
df_focus2['Latitude'] = df_focus2['Latitude'].astype(float)
df_focus2['Longitude'] = df_focus2['Longitude'].astype(float)

# rename to "lat" and "lon"
df_focus2 = df_focus2.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'})

# rename "Offense_Description" to "crime_id"
df_focus2 = df_focus2.rename(columns={'Offense_Description': 'crime_id'})

# only keep the entries where the normalize crime value_count is above 0.01
value_counts = df_focus2['crime_id'].value_counts(normalize=True)
# print(value_counts)
to_keep = value_counts[value_counts > 0.01].index
df_focus2 = df_focus2[df_focus2['crime_id'].isin(to_keep)]

# loop through all unique Offense_Descriptions and assign it an arbitrary number
offense_description_dict = {}
for i, offense in enumerate(df_focus2['crime_id'].unique()):
    offense_description_dict[offense] = i
    
df_focus2['crime_id'] = df_focus2['crime_id'].map(offense_description_dict)

# save the offense_description_dict to json
with open('offense_description_dict.json', 'w') as fp:
    json.dump(offense_description_dict, fp)


# only use the years 2019 to 2022
df_focus2 = df_focus2[df_focus2['Year'] >= 2019]
df_focus2.shape

# save the df to json with comma separated entries encapulated in square brackets
df_focus2.to_json('DataUseCase2-v2.json', orient='records', lines=False)

In [19]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import requests
from bs4 import BeautifulSoup

# Website URL to scrape precinct addresses
url = "https://www.nyc.gov/site/nypd/bureaus/patrol/precincts-landing.page"

# Make an HTTP request to the server
response = requests.get(url)

# Parse the HTML content of the page with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# find the table with the "Address" of the precincts
table = soup.find_all('table')[0]

# Get thet td's with data-label="Address"
tds = table.find_all('td', {'data-label': 'Address'})

# only keep the text of the td's and apend ", New Yowk, NY" to the end
addresses = [td.text + ", New York, NY" for td in tds]

# Initialize Nominatim API
geolocator = Nominatim(user_agent="nypd_crime_data")

# Using rate limiter to avoid overloading the API server
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)


# Dictionary to hold addresses and their coordinates
location_dict = {}

for address in addresses:
    location = geocode(address)
    if location:
        location_dict[address] = [location.latitude, location.longitude]
        print(f"Coordinates for {address}: {location.latitude}, {location.longitude}")
    else:
        location_dict[address] = None
        print(f"Could not find coordinates for {address}")




Coordinates for 16 Ericsson Place, New York, NY: 40.7204814, -74.0071343
Coordinates for 19 Elizabeth Street, New York, NY: 40.7161816, -73.9974633
Could not find coordinates for 233 West 10 Street, New York, NY
Coordinates for 19 1/2 Pitt Street, New York, NY: 40.7162871, -73.9837703
Coordinates for 321 East 5 Street, New York, NY: 42.9304986, -74.1783817
Coordinates for 230 West 20th Street, New York, NY: 40.7486825, -73.34373733091081
Coordinates for 230 East 21st Street, New York, NY: 40.6487106, -73.9598669
Coordinates for 357 West 35th Street, New York, NY: 40.7538182, -73.9949851
Coordinates for 167 East 51st Street, New York, NY: 40.7567621, -73.9707859
Coordinates for 306 West 54th Street, New York, NY: 40.76496015, -73.9850868531467
Coordinates for 153 East 67th Street, New York, NY: 40.767145549999995, -73.96381319536516
Coordinates for 120 West 82nd Street, New York, NY: 40.7840868, -73.97508305171637
Coordinates for 86th St & Transverse Road, New York, NY: 40.7850215, -73.

RateLimiter caught an error, retrying (0/2 tries). Called with (*('168-02 P.O Edward Byrne Ave., New York, NY',), **{}).
Traceback (most recent call last):
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 467, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 462, in _make_request
    httplib_response = conn.getresponse()
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "c:\Users

Could not find coordinates for 168-02 P.O Edward Byrne Ave., New York, NY
Coordinates for 64-2 Catalpa Avenue, New York, NY: 40.7007889, -73.9041025
Coordinates for 92-08 222nd Street, New York, NY: 40.72617005, -73.73512860655313
Coordinates for 103-53 101st Street, New York, NY: 40.68219860000001, -73.83974945
Coordinates for 71-01 Parsons Boulevard, New York, NY: 40.7299492, -73.81062673923094
Coordinates for 5-47 50th Avenue, New York, NY: 40.74303995, -73.95474394328949
Coordinates for 37-05 Union Street, New York, NY: 40.762352199999995, -73.82702044854032
Coordinates for 94-41 43rd Avenue, New York, NY: 40.745239, -73.869946
Coordinates for 45-06 215th Street, New York, NY: 40.76021895, -73.76774854628971
Coordinates for 68-40 Austin Street, New York, NY: 40.722245900000004, -73.85111500966215
Coordinates for 167-02 Baisley Boulevard, New York, NY: 40.6799224, -73.7758415
Coordinates for 34-16 Astoria Boulevard, New York, NY: 40.76927625, -73.91535420061777
Coordinates for 92-15

RateLimiter caught an error, retrying (0/2 tries). Called with (*('116 Main Street, New York, NY',), **{}).
Traceback (most recent call last):
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 467, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 462, in _make_request
    httplib_response = conn.getresponse()
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "c:\Users\jbh\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "c:\Users\jbh\AppData\

Could not find coordinates for 116 Main Street, New York, NY


In [26]:
# print addressses that are None
for address, location in location_dict.items():
    if location is None:
        print(address)

# manually add the missing coordinates
missing_addresses = [(40.733970, -74.005430), (40.819370, -73.944458), (40.706990, -73.792560),(40.51198223739609, -74.25005736108977)]

# add the missing coordinates to the location_dict
for i, address in enumerate(location_dict.keys()):
    if location_dict[address] is None:
        location_dict[address] = missing_addresses.pop(0)

# check if there are any missing coordinates
for address, location in location_dict.items():
    if location is None:
        print(address)

print(location_dict)

# errors caught by viewing the markers on the map, appending the New York, NY to the address is not best practice
location_dict["230 West 20th Street, New York, NY"] = [40.74292291676375, -73.99856790155677]
location_dict["191 Union Street, New York, NY"] = [40.68398405280686, -74.00021197457406]
location_dict["321 East 5 Street, New York, NY"] = [40.68395150938213, -74.00023343224551]
location_dict["211 Union Avenue, New York, NY"] = [40.7064437669646, -73.95075847457295]
location_dict["65 6th Avenue, New York, NY"] = [40.68098252930562, -73.97432521099944]


# save the location_dict to json
with open('precinct_locations.json', 'w') as fp:
    json.dump(location_dict, fp)

{'16 Ericsson Place, New York, NY': [40.7204814, -74.0071343], '19 Elizabeth Street, New York, NY': [40.7161816, -73.9974633], '233 West 10 Street, New York, NY': (40.73397, -74.00543), '19 1/2 Pitt Street, New York, NY': [40.7162871, -73.9837703], '321 East 5 Street, New York, NY': [42.9304986, -74.1783817], '230 West 20th Street, New York, NY': [40.74292291676375, -73.99856790155677], '230 East 21st Street, New York, NY': [40.6487106, -73.9598669], '357 West 35th Street, New York, NY': [40.7538182, -73.9949851], '167 East 51st Street, New York, NY': [40.7567621, -73.9707859], '306 West 54th Street, New York, NY': [40.76496015, -73.9850868531467], '153 East 67th Street, New York, NY': [40.767145549999995, -73.96381319536516], '120 West 82nd Street, New York, NY': [40.7840868, -73.97508305171637], '86th St & Transverse Road, New York, NY': [40.7850215, -73.9685893], '164 East 102nd Street, New York, NY': [40.7891815, -73.9472169], '151 West 100th Street, New York, NY': [40.7964779, -73